# LSTM & AM

In [1]:
import pandas as pd
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#from utils_3A import img_frac, find, postionning, emot_grid, data_video, flatten, cleaning, datafromto
%matplotlib inline
%autosave 60

Autosaving every 60 seconds


## Data preprocessing

In [2]:
def img_frac(img, n, m):
    """
    fractionne l'image en m lignes et n colonnes
    """
    height, width=img.shape[:2]
    x_div=[]
    y_div=[]
    i, j=int(width/n), int(height/m)
    for l in range(1,n):
        x_div.append(l*i)
    for l in range(1,m):
        y_div.append(l*j)

    return (x_div, y_div, img)

def find(x_c, x):
    """
    Position de x_c dans x
    """
    x=[0]+x
    for i in range(len(x)-1):
        if (x_c>=x[i] and x_c<x[i+1]):
            return i
    return i+1

def postionning(frame):
    """
    positionnement de chaque individu dans la grille de l'image
    """
    img_path='Analyse_Json_Stade/Frames 9674/'+str(frame)+'.jpg'
    img=cv2.imread(img_path)
    (x_div, y_div, img)=img_frac(img, n, m)
    base=pd.read_json('Analyse_Json_Stade/AS5I9674/frame_'+str(frame)+'.json')
    base=base[base.type=='person']
    pos_x=[]
    pos_y=[]
    for k in range(len(base)):
        rectangle=base.iloc[k]['data']['rectangle']
        x_c=rectangle['x']+rectangle['width']/2
        y_c=rectangle['y']+rectangle['height']/2
        pos_x.append(find(int(x_c), x_div))
        pos_y.append(find(int(y_c), y_div))
    return (pos_x, pos_y)

def emot_grid(frame, m, n):
    """
    Emotions dans chaque grille de l'image
    """
    (pos_x, pos_y)=postionning(frame)
    base=pd.read_json('Analyse_Json_Stade/AS5I9674/frame_'+str(frame)+'.json')
    base=base[base.type=='person']
    emotions=[[{'anger':0, 'fear':0, 'happiness':0, 'neutral':0, 'sadness':0, 'surprise':0} for _ in range(n)] for _ in range(m)]
    counts=np.zeros((m,n))
    for k in range(len(base)):
        emot=base.iloc[k]['data']['emotion']
        j=pos_x[k]
        i=pos_y[k]
        counts[i][j]+=1
    
        emotions[i][j]['anger']+=emot['anger']
        emotions[i][j]['fear']+=emot['fear']
        emotions[i][j]['happiness']+=emot['happiness']
        emotions[i][j]['neutral']+=emot['neutral']
        emotions[i][j]['sadness']+=emot['sadness']
        emotions[i][j]['surprise']+=emot['surprise']
    for i in range(m):
        for j in range(n):
            if counts[i][j]!=0:
                for sent in ['anger', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']:
                    emotions[i][j][sent]/=counts[i][j]
    return (emotions, counts)

def data_video(video, n, m):
    emotions_video=[]
    counts_video=[]
    i=0
    while os.path.exists('Analyse_Json_Stade/Frames '+str(video)+'/%d.jpg' % i):   
        (emotions, counts)=emot_grid(i, m, n)
        emotions_video.append(emotions)
        counts_video.append(counts)
        i+=1
        if i%500==0:
            print(i)
    return (emotions_video, counts_video)

def flatten(l):
    flatten_emotions=[0 for i in range(len(l))]
    for k in range(len(l)):
        flatten_emotions[k]=[list(l[k][i][j].values()) for i in range(m) for j in range(n)] 
    return flatten_emotions

def cleaning(l):
    for k in range(len(l)):
        for m in range(len(l[k])):
            if sum(l[k][m])==0:
                l[k][m]=[0.01, 0.01, 0.01, 0.95, 0.01, 0.01]#neutral
    return l
    
def datafromto(l, fps, start, end):
    n_i=start*fps
    n_f=end*fps
    x=l[n_i:n_f]
    x=torch.tensor(np.array(x), requires_grad=False)
    x=x.type(torch.FloatTensor)
    return x

def metric(a,b):
    return int(a.index(min(a))==b.index(min(b)))

In [3]:
n, m=15, 10
video='9674'
T_enc=40
T_dec=10
fps=24

(emotions_video, counts_video)=data_video(video, n, m)
flatten_emotions=flatten(emotions_video)
flatten_emotions=cleaning(flatten_emotions)

tensor_enc=datafromto(flatten_emotions, fps, 0, T_enc)
tensor_dec=datafromto(flatten_emotions, fps, T_enc, T_enc+T_dec)

500
1000
1500


## Modélisation

### The Seq2Seq Model


A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

<img src=https://pytorch.org/tutorials/_images/seq2seq.png>

### The Encoder

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence.

In [5]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, hidden_size3):
        super(EncoderRNN, self).__init__()
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.hidden_size3 = hidden_size3

        self.lstm1=nn.LSTMCell(input_size, hidden_size1)
        self.lstm2=nn.LSTMCell(hidden_size1, hidden_size2)
        self.lstm3=nn.LSTMCell(hidden_size2, hidden_size3)
        
        self.lin1=torch.nn.Linear(hidden_size1, hidden_size1)
        self.lin2=torch.nn.Linear(hidden_size2, hidden_size2)
        self.lin3=torch.nn.Linear(hidden_size3, input_size)
        
    def forward(self, input, hx1,cx1, hx2,cx2, hx3,cx3):
        hx1, cx1 = self.lstm1(input, (hx1, cx1))
        hx2, cx2 = self.lstm2(F.tanh(self.lin1(hx1)), (hx2, cx2))
        hx3, cx3 = self.lstm3(F.tanh(self.lin2(hx2)), (hx3, cx3))
        output=F.tanh(self.lin3(hx3))
        output = F.log_softmax(output)#softmax
        return output, hx1,cx1, hx2,cx2, hx3,cx3

    def initHidden(self, batch_size):
        #initialization of c(cell's state) et h(hidden state) h=Otanh(c)
        return [(torch.zeros(batch_size, self.hidden_size1, device=device),torch.zeros(batch_size, self.hidden_size1, device=device)),
               (torch.zeros(batch_size, self.hidden_size2, device=device),torch.zeros(batch_size, self.hidden_size2, device=device)),
               (torch.zeros(batch_size, self.hidden_size3, device=device),torch.zeros(batch_size, self.hidden_size3, device=device))]
    

#### Test encodeur

input_size=6
hidden_size1=10
hidden_size2=20
hidden_size3=10

encoder=EncoderRNN(input_size, hidden_size1,hidden_size2, hidden_size3)
X_enc=tensor_enc[0]

batch_size=X_enc.shape[0]
init_enc= encoder.initHidden(batch_size)
hx1_enc, cx1_enc=init_enc[0]
hx2_enc, cx2_enc=init_enc[1]
hx3_enc, cx3_enc=init_enc[2]

output_enc, hx1_enc,cx1_enc, hx2_enc,cx2_enc, hx3_enc,cx3_enc=encoder(X_enc, hx1_enc, cx1_enc, hx2_enc, cx2_enc, 
                                                                      hx3_enc, cx3_enc)

output_enc[0]

### Attention Decoder

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of values.
Simple Decoder


In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input and hidden state, 
and the first hidden state is the context vector (the encoder's
last hidden state).

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

<img src=https://i.imgur.com/1152PYf.png>

In [24]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.hidden_size3 = hidden_size3
        self.output_size = output_size

        self.attn = nn.Linear(output_size + hidden_size1, 1)
        self.attn_combine = nn.Linear(self.output_size*2, self.output_size)
        
        self.lstm1=nn.LSTMCell(output_size, hidden_size1)
        self.lstm2=nn.LSTMCell(hidden_size1, hidden_size2)
        self.lstm3=nn.LSTMCell(hidden_size2, hidden_size3)
        
        self.lin1=torch.nn.Linear(hidden_size1, hidden_size1)
        self.lin2=torch.nn.Linear(hidden_size2, hidden_size2)
        self.lin3=torch.nn.Linear(hidden_size3, output_size)

    def forward(self, input, encoder_outputs, hx1,cx1, hx2,cx2, hx3,cx3):
        attns=[]
        for k in range(len(encoder_outputs)):
            x=torch.cat((encoder_outputs[k],hx1), dim=1) #[batch_size, input_size+hidden_size]
            x=self.attn(x) #[batch_size,1]
            attns.append(x)
            
        attns=torch.transpose(torch.cat(tuple(attns),1),0,1) #[input_lenth, batch_size,1]    
        attn_weights = torch.transpose(F.softmax(attns, dim=0),0,1) #[input_lenth, batch_size] 
        attn_applied=[]
        for k in range(encoder_outputs.shape[1]):
            x=torch.matmul(torch.transpose(torch.transpose(encoder_outputs, 0, 1)[k],0,1), 
                   attn_weights[k].view(attn_weights[k].shape[0],1))
            attn_applied.append(x)
        attn_applied=torch.transpose(torch.cat(tuple(attn_applied),1),0,1) #[batch_size, input_size]

        ci = torch.cat((input, attn_applied), 1) #[batch_size, 2*input_size]
        ci = self.attn_combine(ci) #[batch_size, input_size]
        ci = F.relu(ci)
       
        hx1, cx1 = self.lstm1(ci, (hx1, cx1))
        hx2, cx2 = self.lstm2(F.tanh(self.lin1(hx1)), (hx2, cx2))
        hx3, cx3 = self.lstm3(F.tanh(self.lin2(hx2)), (hx3, cx3))
        output=F.tanh(self.lin3(hx3))
        output = F.log_softmax(output)#softmax
        return output, attn_weights, hx1,cx1, hx2,cx2, hx3,cx3    

    def initHidden(self, batch_size):
        #initialization of c(cell's state) et h(hidden state) h=Otanh(c)
        return [(torch.zeros(batch_size, self.hidden_size1, device=device),torch.zeros(batch_size, self.hidden_size1, device=device)),
               (torch.zeros(batch_size, self.hidden_size2, device=device),torch.zeros(batch_size, self.hidden_size2, device=device)),
               (torch.zeros(batch_size, self.hidden_size3, device=device),torch.zeros(batch_size, self.hidden_size3, device=device))]
   

#### Test decodeur

input_size=6
hidden_size1=10
hidden_size2=20
hidden_size3=10
output_size=input_size

encoder=EncoderRNN(input_size, hidden_size1,hidden_size2, hidden_size3)
X_enc=tensor_enc[0]

batch_size=X_enc.shape[0]
init_enc= encoder.initHidden(batch_size)
hx1_enc, cx1_enc=init_enc[0]
hx2_enc, cx2_enc=init_enc[1]
hx3_enc, cx3_enc=init_enc[2]

output_enc, hx1_enc,cx1_enc, hx2_enc,cx2_enc, hx3_enc,cx3_enc=encoder(X_enc, hx1_enc, cx1_enc, hx2_enc, cx2_enc, 
                                                                      hx3_enc, cx3_enc)


decoder=AttnDecoderRNN(hidden_size1, hidden_size2, hidden_size3, output_size)
X_dec=tensor_dec[0]
encoder_outputs=tensor_enc

hx1_dec,cx1_dec=hx1_enc,cx1_enc
hx2_dec,cx2_dec=hx2_enc,cx2_enc
hx3_dec,cx3_dec=hx3_enc,cx3_enc

output_dec, attn_weights,x1_dec,cx1_dec, hx2_dec,cx2_dec, hx3_dec,cx3_dec=decoder(X_dec, encoder_outputs,hx1_dec, cx1_dec, hx2_dec, cx2_dec, 
                                                                     hx3_dec, cx3_dec)

output_dec[0]

## Training the Model

To train we run the input through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

In [28]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    batch_size=input_tensor.shape[1]
    init_enc= encoder.initHidden(batch_size)
    hx1_enc, cx1_enc=init_enc[0]
    hx2_enc, cx2_enc=init_enc[1]
    hx3_enc, cx3_enc=init_enc[2]

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.shape[0]
    target_length = target_tensor.shape[0]
    
    #Setting new minibatchs
    minibatch_enc=random.randint(10,20)*24
    minibatch_dec=random.randint(5,10)*24
    
    input_tensor=input_tensor[input_length-minibatch_enc:input_length]
    target_tensor=target_tensor[:minibatch_dec]
    
    input_length = input_tensor.shape[0]
    target_length = target_tensor.shape[0]
    
    loss = 0
    metrics=0
    encoder_outputs=[]
    for ei in range(input_length):
        output_enc, hx1_enc,cx1_enc, hx2_enc,cx2_enc, hx3_enc,cx3_enc=encoder(input_tensor[ei], hx1_enc, cx1_enc, 
                                                                              hx2_enc, cx2_enc, hx3_enc, cx3_enc)
        encoder_outputs.append(output_enc)
    encoder_outputs=torch.cat(tuple(encoder_outputs),0).view(input_tensor.shape[0],input_tensor.shape[1],input_tensor.shape[2])        
    hx1_dec,cx1_dec=hx1_enc,cx1_enc
    hx2_dec,cx2_dec=hx2_enc,cx2_enc
    hx3_dec,cx3_dec=hx3_enc,cx3_enc
    attns=[]
    # Teacher forcing: Feed the target as the next input
    for di in range(target_length):
        output_dec, attn_weights, hx1_dec,cx1_dec, hx2_dec,cx2_dec, hx3_dec,cx3_dec=decoder(target_tensor[di], encoder_outputs, hx1_dec, cx1_dec, 
                                                                             hx2_dec, cx2_dec,hx3_dec, cx3_dec)
        loss += criterion(output_dec, target_tensor[di])
        attns.append(attn_weights)
        a=[metric(list(output_dec[k]), list(target_tensor[di][k])) for k in range(output_dec.shape[0])]
        metrics+=sum(a)/output_dec.shape[0]
        
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return (loss.item()/target_length, metrics/target_length)

In [29]:
import time
import math

def asMinutes(s):
    """
    Transforme les secondes en minutes et secondes
    """
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [30]:
def trainIters(input_tensor, target_tensor, encoder, decoder, n_iters, print_every, learning_rate):
    start = time.time()
    print_loss_total = 0  # Reset every print_every
    print_metrics_total = 0
    
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.MSELoss()
    batch_size=input_tensor.shape[0]

    for iter in range(1, n_iters + 1):
        #penser à la permutation
        loss, metrics = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        print_metrics_total += metrics
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_metrics_avg = print_metrics_total / print_every
            print_loss_total = 0
            print_metrics_total = 0
            
            print('# %s (%d %d%%) (%.4f %.2f%%)' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg, print_metrics_avg*100))

With all these helper functions in place, we can actually initialize a network and start training.



In [31]:
input_size=6
hidden_size1=10
hidden_size2=20
hidden_size3=10
output_size=input_size

n_iters=20000
print_every=1000
learning_rate=0.01

input_tensor=tensor_enc
target_tensor=torch.tensor(np.log(np.array(tensor_dec)), requires_grad=False).type(torch.FloatTensor)

encoder=EncoderRNN(input_size, hidden_size1, hidden_size2, hidden_size3).to(device)
decoder=AttnDecoderRNN(hidden_size1, hidden_size2, hidden_size3, output_size).to(device)

In [32]:
trainIters(input_tensor, target_tensor, encoder, decoder, n_iters, print_every, learning_rate)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


# 13m 37s (- 0m 0s) (1 100%) (1.2895 10.71%)
